In [15]:
# libraries and set up
import requests
import json
from datetime import datetime
import pprint

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from pandas import Series, DataFrame


In [16]:
# 1. Make an API call and store the response.
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?'
query = "Bezos"
query_url = url + 'api-key=t3pOI4nLY54cY1AebsGImGdhXAUKAwGt' + "&q=" + query
print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=t3pOI4nLY54cY1AebsGImGdhXAUKAwGt&q=Bezos


In [19]:
# 2. Get Articles
bezos_articles = requests.get(query_url).json()

pprint.pprint(bezos_articles)

{'copyright': 'Copyright (c) 2022 The New York Times Company. All Rights '
              'Reserved.',
 'response': {'docs': [{'_id': 'nyt://article/7809b909-a012-50f3-a88c-539e1762e9fd',
                        'abstract': 'Starting in January, space tourists will '
                                    'not receive a participation trophy for '
                                    'flying to space. But everyone will be on '
                                    'the honor roll.',
                        'byline': {'organization': None,
                                   'original': 'By Joey Roulette',
                                   'person': [{'firstname': 'Joey',
                                               'lastname': 'Roulette',
                                               'middlename': None,
                                               'organization': '',
                                               'qualifier': None,
                                               'rank': 1,

                                        'url': 'images/2021/11/22/multimedia/22bezos-obama/22bezos-obama-threeByTwoMediumAt2X-v3.jpg',
                                        'width': 1500},
                                       {'caption': None,
                                        'credit': None,
                                        'crop_name': 'threeByTwoSmallAt2X',
                                        'height': 400,
                                        'legacy': {},
                                        'rank': 0,
                                        'subType': 'threeByTwoSmallAt2X',
                                        'subtype': 'threeByTwoSmallAt2X',
                                        'type': 'image',
                                        'url': 'images/2021/11/22/multimedia/22bezos-obama/22bezos-obama-threeByTwoSmallAt2X-v3.jpg',
                                        'width': 600},
                                       {'caption': None,
         

                                        'width': 6240},
                                       {'caption': None,
                                        'credit': None,
                                        'crop_name': 'hpLarge',
                                        'height': 287,
                                        'legacy': {},
                                        'rank': 0,
                                        'subType': 'hpLarge',
                                        'subtype': 'hpLarge',
                                        'type': 'image',
                                        'url': 'images/2021/12/09/multimedia/09db-newsletter-stoxx/09db-newsletter-stoxx-hpLarge.jpg',
                                        'width': 511},
                                       {'caption': None,
                                        'credit': None,
                                        'crop_name': 'largeWidescreen573',
                                        'height'

                                       {'caption': None,
                                        'credit': None,
                                        'crop_name': 'superJumbo',
                                        'height': 1366,
                                        'legacy': {},
                                        'rank': 0,
                                        'subType': 'superJumbo',
                                        'subtype': 'superJumbo',
                                        'type': 'image',
                                        'url': 'images/2021/11/23/multimedia/23xp-strahan-1/23xp-strahan-1-superJumbo.jpg',
                                        'width': 2048},
                                       {'caption': None,
                                        'credit': None,
                                        'crop_name': 'blog225',
                                        'height': 150,
                                        'legacy': {},
      

In [20]:
bezos_list = bezos_articles["response"]["docs"]
pprint.pprint(bezos_list)

[{'_id': 'nyt://article/7809b909-a012-50f3-a88c-539e1762e9fd',
  'abstract': 'Starting in January, space tourists will not receive a '
              'participation trophy for flying to space. But everyone will be '
              'on the honor roll.',
  'byline': {'organization': None,
             'original': 'By Joey Roulette',
             'person': [{'firstname': 'Joey',
                         'lastname': 'Roulette',
                         'middlename': None,
                         'organization': '',
                         'qualifier': None,
                         'rank': 1,
                         'role': 'reported',
                         'title': None}]},
  'document_type': 'article',
  'headline': {'content_kicker': None,
               'kicker': None,
               'main': 'Jeff Bezos Is Getting Astronaut Wings. But Soon, the '
                       'F.A.A. Won’t Award Them.',
               'name': None,
               'print_headline': 'Space Tourist or Astron

                  'url': 'images/2021/11/22/multimedia/22bezos-obama/22bezos-obama-videoSixteenByNine600-v3.jpg',
                  'width': 600},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'videoSixteenByNine540',
                  'height': 304,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'videoSixteenByNine540',
                  'subtype': 'videoSixteenByNine540',
                  'type': 'image',
                  'url': 'images/2021/11/22/multimedia/22bezos-obama/22bezos-obama-videoSixteenByNine540-v3.jpg',
                  'width': 540},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'videoSixteenByNine495',
                  'height': 278,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'videoSixteenByNine495',
                  'subtype': 'videoSixteenByNine495',
                 

                 {'caption': None,
                  'credit': None,
                  'crop_name': 'articleInline',
                  'height': 127,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'articleInline',
                  'subtype': 'articleInline',
                  'type': 'image',
                  'url': 'images/2021/12/09/multimedia/09db-newsletter-stoxx/09db-newsletter-stoxx-articleInline.jpg',
                  'width': 190},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'hpSmall',
                  'height': 109,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'hpSmall',
                  'subtype': 'hpSmall',
                  'type': 'image',
                  'url': 'images/2021/12/09/multimedia/09db-newsletter-stoxx/09db-newsletter-stoxx-hpSmall.jpg',
                  'width': 163},
                 {'caption': None,
         

                 {'caption': None,
                  'credit': None,
                  'crop_name': 'hpLarge',
                  'height': 287,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'hpLarge',
                  'subtype': 'hpLarge',
                  'type': 'image',
                  'url': 'images/2021/11/23/multimedia/23xp-strahan-1/23xp-strahan-1-hpLarge.jpg',
                  'width': 511},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'largeWidescreen573',
                  'height': 322,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'largeWidescreen573',
                  'subtype': 'largeWidescreen573',
                  'type': 'image',
                  'url': 'images/2021/11/23/multimedia/23xp-strahan-1/23xp-strahan-1-largeWidescreen573.jpg',
                  'width': 573},
                 {'caption': None,
                 